## StarTracker WCS solutions - 20-Feb-23

Craig Lage - Using the boresights as the reported location

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as pf
from astropy.time import Time, TimeDelta
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5, SkyCoord
import astropy.units as u
from lsst.obs.lsst.translators.latiss import AUXTEL_LOCATION
from lsst.obs.lsst.translators.lsst import SIMONYI_LOCATION

import lsst.afw.image as afwImage
from lsst.summit.utils.utils import starTrackerFileToExposure
from lsst.summit.utils.astrometry.anet import CommandLineSolver
from lsst.summit.utils.astrometry.utils import runCharactierizeImage, filterSourceCatOnBrightest

In [ ]:
solver = CommandLineSolver('/project/shared/ref_cats/astrometry_net/')

In [ ]:
#cameras = [['Wide', 101], ['Narrow', 102]]
cameras = [['Narrow', 102], ['Fast', 103]]
dates = [20230321]
for [camera, num] in cameras:
    for date in dates:
        year = int(date/10000)
        month = int((date - 10000 * year)/100)
        day = int((date - 10000 * year - 100 * month))
        path = f"/project/GenericCamera/{num}/{year}/{month:02}/{day:02}/"
        outfile = open(f"/scratch/cslage/starTracker/StarTracker_Summary_Boresight_{camera}_{date}.txt", "w")
        outfile.write("SeqNum\tRA(header)\tDec(header)\tEl(header)\tAz(header)\tRA(Solve)\tDec(Solve)\tDeltaRA\tDeltaDec\tEl(solve)\tAz(solve)\tDeltaEl\tDeltaAz\tRot\tRMS(asec)\n")
        for seqNum in range(1400, 1401):
            try:    
                filename = path + f"GC{num}_O_{date}_{seqNum:06}.fits"
                exp = starTrackerFileToExposure(filename)
                results = runCharactierizeImage(exp, 5, 25)
                filteredCat = filterSourceCatOnBrightest(results.sourceCat, 0.2, maxSources=100)
                if camera == 'Narrow':
                    # Need to loosen percentageScaleError because header SECPIX is wrong
                    solverResults = solver.run(exp, filteredCat, percentageScaleError=60.0, isWideField=True)
                    boresight = (1898.10, 998.47)
                if camera == 'Wide':
                    solverResults = solver.run(exp, filteredCat, isWideField=True)
                    boresight = (1560.85, 1257.15)
                newWcs = solverResults.wcs
                rms_error = solverResults.rmsErrorArsec
                boresight_RaDec = newWcs.pixelToSky(boresight[0], boresight[1])
                raSolve = boresight_RaDec.getRa().asDegrees()
                decSolve = boresight_RaDec.getDec().asDegrees()
            except:
                print(f"{camera}, {date}_{seqNum} Failed!")
                continue
            hdulist = pf.open(filename)
            header = hdulist[0].header
            raStart = header["RASTART"]
            decStart = header["DECSTART"]
            azHdr = (header["AZSTART"] + header["AZEND"]) / 2.0
            azHdr = azHdr % 360
            elHdr = (header["ELSTART"] + header["ELEND"]) / 2.0
            rotpa = header["ROTPA"]
            rot = newWcs.getRelativeRotationToWcs(exp.getWcs()).asDegrees()
            #time = Time(header["DATE-OBS"], scale='tai')
            time = Time((Time(header["DATE-BEG"], scale='tai').unix_tai + Time(header["DATE-END"], scale='tai').unix_tai)/2.0, format='unix_tai', scale='tai')
            altAz = AltAz(obstime=time, location=SIMONYI_LOCATION)
            skyLocation = SkyCoord(raSolve * u.deg, decSolve * u.deg)
            obsAltAz = skyLocation.transform_to(altAz)
            az = obsAltAz.az.deg
            az = az % 360
            el = obsAltAz.alt.deg
            deltaAz = (azHdr - az) * 3600.0
            deltaEl = (elHdr - el) * 3600.0
            deltaRA = (raStart - raSolve) * 3600.0
            deltaDec = (decStart - decSolve) * 3600.0
            print(f"{camera}, {date}_{seqNum}, {deltaAz:.5f}, {deltaEl:.5f}")
            outfile.write(f"{seqNum}\t{raStart:.5f}\t{decStart:.5f}\t{elHdr:.5f}\t{azHdr:.5f}\t{raSolve:.5f}\t{decSolve:.5f}\t{deltaRA:.5f}\t{deltaDec:.5f}\t{el:.5f}\t{az:.5f}\t{deltaEl:.5f}\t{deltaAz:.5f}\t{rot:.5f}\t{rms_error:.5f}\n")
        outfile.close()
    

In [ ]:
Time(header["DATE-BEG"], scale='tai')

In [ ]:
solverResults.wcs.isFlipped

In [ ]:
header["DATE-BEG"]